<a href="https://colab.research.google.com/github/ENRICOROBLESDELRIO/data_boot_camp_final_project/blob/cgzverduzco_branch/Machine%20Learning%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import data**

In [2]:
# Libraries and dependencies

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import math
from sklearn.metrics import accuracy_score
from google.colab import files
from sqlalchemy import create_engine
import psycopg2

## Import for models
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# Google Cloud SQL connexion:
# Code taken from: https://stackoverflow.com/questions/53180140/how-to-access-cloud-sql-from-google-colab
!gcloud auth login
!gcloud config set project endeavor-intelligence-unit
!gcloud sql instances describe finalprojectdb


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Ekp2XeHKNuG3UTFE2CSDM9NLL5lVpX&prompt=consent&access_type=offline&code_challenge=ST2r-b2i6k7POX99Nj4Rw0C31Ck0oGfQx19RU3qpZZU&code_challenge_method=S256

Enter verification code: 4/1AX4XfWh7s2tA_B0iajuCrIsKkBSqSUuuweqlHuECbzoqe6vSabnWhw2JVk4

You are now logged in as [c.gzverduzco@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
ERROR: (gcloud.sql.instances.describe) There was no ins

In [4]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy

# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="endeavor-intelligence-unit:northamerica-northeast2:finalprojectdb"=tcp:5432 &
!sleep 30s

--2021-11-13 17:15:38--  https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64
Resolving dl.google.com (dl.google.com)... 108.177.119.91, 108.177.119.190, 108.177.119.93, ...
Connecting to dl.google.com (dl.google.com)|108.177.119.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16638816 (16M) [application/octet-stream]
Saving to: ‘cloud_sql_proxy’

cloud_sql_proxy     100%[===================>]  15.87M  --.-KB/s    in 0.06s   

2021-11-13 17:15:38 (267 MB/s) - ‘cloud_sql_proxy’ saved [16638816/16638816]

nohup: appending output to 'nohup.out'


In [5]:
#Checking the proxy´s logs 
!cat nohup.out

2021/11/13 17:15:38 current FDs rlimit set to 1048576, wanted limit is 8500. Nothing to do here.
2021/11/13 17:15:40 errors parsing config:
	googleapi: Error 403: The client is not authorized to make this request., notAuthorized


In [6]:
# Connexion throught psycopg2 Google Cloud SQL data base

conn = psycopg2.connect(
    host='*******', port='********', database="*********",
    user="*******", password="*********")

#Rounds

In [8]:
# Query to connect with SQL data base
SQL_Query = pd.read_sql_query('''select * FROM "Rounds" ''', conn)
rounds_df = pd.DataFrame(SQL_Query)

In [9]:
# Data base of VC Rounds in entrepreneurs 
rounds_df.head(20)

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team
0,3,E-Commerce,Seed,2017,33.0,8,3,1,0,1,2017,1400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,SaaS,Seed,2018,10.0,0,2,0,0,1,2018,10500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,SaaS,Angel,2015,11.0,14,1,1,0,1,2015,200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,IT services,Seed,2016,10.0,0,1,0,0,1,2016,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23,E-Commerce,Seed,2013,1001.0,0,1,1,1,0,2013,150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,23,E-Commerce,Seed,2020,1001.0,0,3,1,1,0,2020,2500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,23,E-Commerce,Seed,2020,1001.0,0,4,1,1,0,2020,150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,23,E-Commerce,Series A,2020,1001.0,0,0,1,0,0,2020,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,23,E-Commerce,Series B,2021,1001.0,0,2,1,1,0,2021,40000000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,33,SaaS,Accelerator/Incubator,2013,6.0,0,1,1,1,0,2013,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# data types
rounds_df.dtypes

id_company                     int64
category                      object
deal_type                     object
foundation_year                int64
number_employees             float64
increase_employees             int64
number_investors               int64
current_status                 int64
foreign_vc                     int64
local_vc                       int64
round_year                     int64
round_amount                 float64
support_company              float64
manufacture_equipment        float64
develop_new_tech             float64
grow_company                 float64
add_new_services             float64
develop_business             float64
financial_banking            float64
develop_products_services    float64
marketing_strategies         float64
expand_internationality      float64
consolidate_company          float64
increase_team                float64
dtype: object

In [11]:
rounds_df.isna().sum()

id_company                   0
category                     0
deal_type                    0
foundation_year              0
number_employees             7
increase_employees           0
number_investors             0
current_status               0
foreign_vc                   0
local_vc                     0
round_year                   0
round_amount                 0
support_company              5
manufacture_equipment        5
develop_new_tech             5
grow_company                 5
add_new_services             5
develop_business             5
financial_banking            5
develop_products_services    5
marketing_strategies         5
expand_internationality      5
consolidate_company          5
increase_team                5
dtype: int64

In [12]:
rounds_df[rounds_df['current_status'].isnull()]


,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team


In [13]:
rounds_df.count()

id_company                   1236
category                     1236
deal_type                    1236
foundation_year              1236
number_employees             1229
increase_employees           1236
number_investors             1236
current_status               1236
foreign_vc                   1236
local_vc                     1236
round_year                   1236
round_amount                 1236
support_company              1231
manufacture_equipment        1231
develop_new_tech             1231
grow_company                 1231
add_new_services             1231
develop_business             1231
financial_banking            1231
develop_products_services    1231
marketing_strategies         1231
expand_internationality      1231
consolidate_company          1231
increase_team                1231
dtype: int64

In [14]:
len(rounds_df.axes[0])

1236

In [15]:
# Drop the null columns where all values are null
rounds_df = rounds_df.dropna(axis='columns', how='all')
rounds_df.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team
0,3,E-Commerce,Seed,2017,33.0,8,3,1,0,1,2017,1400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,SaaS,Seed,2018,10.0,0,2,0,0,1,2018,10500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,SaaS,Angel,2015,11.0,14,1,1,0,1,2015,200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,IT services,Seed,2016,10.0,0,1,0,0,1,2016,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23,E-Commerce,Seed,2013,1001.0,0,1,1,1,0,2013,150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Encoding labels
le = LabelEncoder()
db_rounds_Encode = rounds_df.copy()
db_rounds_Encode['deal_type'] = le.fit_transform(db_rounds_Encode['deal_type'])

# Get the name of the columns
columnsName = db_rounds_Encode.columns.tolist()
#columnsName.remove('rounds_funding_factor')

# convert variables to numerical
for column in columnsName:
  db_rounds_Encode[column] = pd.to_numeric(db_rounds_Encode[column], errors='coerce').fillna(0).astype(np.int64)

db_rounds_Encode.head()

,id_company,category,deal_type,foundation_year,number_employees,increase_employees,number_investors,current_status,foreign_vc,local_vc,round_year,round_amount,support_company,manufacture_equipment,develop_new_tech,grow_company,add_new_services,develop_business,financial_banking,develop_products_services,marketing_strategies,expand_internationality,consolidate_company,increase_team
0,3,0,10,2017,33,8,3,1,0,1,2017,1400000,0,0,0,0,0,0,0,0,0,0,0,0
1,7,0,10,2018,10,0,2,0,0,1,2018,10500,0,0,0,0,0,0,0,0,0,0,0,0
2,10,0,1,2015,11,14,1,1,0,1,2015,200000,0,0,0,0,0,0,0,0,0,0,0,0
3,13,0,10,2016,10,0,1,0,0,1,2016,50000,0,0,0,0,0,0,0,0,0,0,0,0
4,23,0,10,2013,1001,0,1,1,1,0,2013,150000,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# data types
db_rounds_Encode.dtypes

id_company                   int64
category                     int64
deal_type                    int64
foundation_year              int64
number_employees             int64
increase_employees           int64
number_investors             int64
current_status               int64
foreign_vc                   int64
local_vc                     int64
round_year                   int64
round_amount                 int64
support_company              int64
manufacture_equipment        int64
develop_new_tech             int64
grow_company                 int64
add_new_services             int64
develop_business             int64
financial_banking            int64
develop_products_services    int64
marketing_strategies         int64
expand_internationality      int64
consolidate_company          int64
increase_team                int64
dtype: object

In [18]:
db_rounds_Encode.isna().sum()

id_company                   0
category                     0
deal_type                    0
foundation_year              0
number_employees             0
increase_employees           0
number_investors             0
current_status               0
foreign_vc                   0
local_vc                     0
round_year                   0
round_amount                 0
support_company              0
manufacture_equipment        0
develop_new_tech             0
grow_company                 0
add_new_services             0
develop_business             0
financial_banking            0
develop_products_services    0
marketing_strategies         0
expand_internationality      0
consolidate_company          0
increase_team                0
dtype: int64

In [19]:
db_rounds_Encode.count()

id_company                   1236
category                     1236
deal_type                    1236
foundation_year              1236
number_employees             1236
increase_employees           1236
number_investors             1236
current_status               1236
foreign_vc                   1236
local_vc                     1236
round_year                   1236
round_amount                 1236
support_company              1236
manufacture_equipment        1236
develop_new_tech             1236
grow_company                 1236
add_new_services             1236
develop_business             1236
financial_banking            1236
develop_products_services    1236
marketing_strategies         1236
expand_internationality      1236
consolidate_company          1236
increase_team                1236
dtype: int64

#FOUNDERS

In [20]:
# Query to connect with SQL data base from Founders Demographics 
SQL_Query = pd.read_sql_query('''select * FROM "Founders" ''', conn)
founders_df = pd.DataFrame(SQL_Query)

In [21]:
# Data base of Founders Demographics  
founders_df.head(20)

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,"Ciudad de México, México",1,1,0,0,0,0,0,1.0,2020.0,12.0,31.0,32.0,13.0,4,M,0,0,0
1,2,23,"Ciudad de México, México",0,1,0,0,0,0,0,1.0,2014.0,4.0,25.0,32.0,11.0,2,M,0,0,0
2,3,141,México,0,1,0,0,0,0,0,0.0,2015.0,0.0,22.0,28.0,6.0,1,M,0,1,0
3,4,152,"Ciudad de México, México",0,0,0,0,1,0,1,1.0,2017.0,6.0,28.0,32.0,10.0,3,M,1,1,1
4,5,155,México,1,1,0,0,0,0,0,0.0,2015.0,10.0,27.0,33.0,16.0,1,M,1,0,0
5,6,167,None,0,1,0,0,0,0,0,1.0,2003.0,0.0,NaN,41.0,18.0,1,M,0,0,0
6,7,185,"Ciudad de México, México",0,0,0,0,0,0,0,1.0,2021.0,20.0,41.0,41.0,20.0,4,M,1,0,0
7,8,233,"Ciudad de México, México",1,1,0,0,0,0,1,1.0,2016.0,13.0,34.0,39.0,18.0,2,F,0,1,1
8,9,1799,None,1,1,1,0,0,0,0,1.0,2013.0,25.0,54.0,62.0,33.0,1,M,0,1,0
9,10,284,"Ciudad de México, México",0,0,0,0,1,0,0,1.0,2016.0,2.0,24.0,29.0,7.0,1,M,0,0,0


In [22]:
# data types
founders_df.dtypes

id_founder                               int64
id_company                               int64
general_location                        object
manager_exp                              int64
clevel_exp                               int64
engineering_exp                          int64
sales_exp                                int64
finance_exp                              int64
human_capital_exp                        int64
consulting_exp                           int64
founder_exp                            float64
foundation_year                        float64
years_of_experience_as_entreprenuer    float64
age_of_first_entrepreneurship          float64
age                                    float64
years_of_experience                    float64
number_of_startups                       int64
gender                                  object
foreing_studies                          int64
foreing_work_experience                  int64
foreing_founder                          int64
dtype: object

In [23]:
founders_df.isna().sum()

id_founder                              0
id_company                              0
general_location                       11
manager_exp                             0
clevel_exp                              0
engineering_exp                         0
sales_exp                               0
finance_exp                             0
human_capital_exp                       0
consulting_exp                          0
founder_exp                             1
foundation_year                         7
years_of_experience_as_entreprenuer     7
age_of_first_entrepreneurship          74
age                                     6
years_of_experience                     6
number_of_startups                      0
gender                                  0
foreing_studies                         0
foreing_work_experience                 0
foreing_founder                         0
dtype: int64

In [24]:
founders_df.count()


id_founder                             640
id_company                             640
general_location                       629
manager_exp                            640
clevel_exp                             640
engineering_exp                        640
sales_exp                              640
finance_exp                            640
human_capital_exp                      640
consulting_exp                         640
founder_exp                            639
foundation_year                        633
years_of_experience_as_entreprenuer    633
age_of_first_entrepreneurship          566
age                                    634
years_of_experience                    634
number_of_startups                     640
gender                                 640
foreing_studies                        640
foreing_work_experience                640
foreing_founder                        640
dtype: int64

In [25]:
# Drop the null columns where all values are null
founders_df = founders_df.dropna(axis='columns', how='all')
founders_df.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,"Ciudad de México, México",1,1,0,0,0,0,0,1.0,2020.0,12.0,31.0,32.0,13.0,4,M,0,0,0
1,2,23,"Ciudad de México, México",0,1,0,0,0,0,0,1.0,2014.0,4.0,25.0,32.0,11.0,2,M,0,0,0
2,3,141,México,0,1,0,0,0,0,0,0.0,2015.0,0.0,22.0,28.0,6.0,1,M,0,1,0
3,4,152,"Ciudad de México, México",0,0,0,0,1,0,1,1.0,2017.0,6.0,28.0,32.0,10.0,3,M,1,1,1
4,5,155,México,1,1,0,0,0,0,0,0.0,2015.0,10.0,27.0,33.0,16.0,1,M,1,0,0


In [26]:
# Encoding labels
le = LabelEncoder()
db_founders_Encode = founders_df.copy()
db_founders_Encode['general_location'] = db_founders_Encode['general_location'].fillna("nan", inplace = True) # Aqui Luis
db_founders_Encode['general_location'] = le.fit_transform(db_founders_Encode['general_location'])
db_founders_Encode['gender'] = le.fit_transform(db_founders_Encode['gender'])
#db_founders_Encode['category'] = le.fit_transform(db_founders_Encode['category'])

# Get the name of the columns
columnsName = db_founders_Encode.columns.tolist()

# convert variables to numerical
for column in columnsName:
  db_founders_Encode[column] = pd.to_numeric(db_founders_Encode[column], errors='coerce').fillna(0).astype(np.int64)

db_founders_Encode.head()

,id_founder,id_company,general_location,manager_exp,clevel_exp,engineering_exp,sales_exp,finance_exp,human_capital_exp,consulting_exp,founder_exp,foundation_year,years_of_experience_as_entreprenuer,age_of_first_entrepreneurship,age,years_of_experience,number_of_startups,gender,foreing_studies,foreing_work_experience,foreing_founder
0,1,7,0,1,1,0,0,0,0,0,1,2020,12,31,32,13,4,1,0,0,0
1,2,23,0,0,1,0,0,0,0,0,1,2014,4,25,32,11,2,1,0,0,0
2,3,141,0,0,1,0,0,0,0,0,0,2015,0,22,28,6,1,1,0,1,0
3,4,152,0,0,0,0,0,1,0,1,1,2017,6,28,32,10,3,1,1,1,1
4,5,155,0,1,1,0,0,0,0,0,0,2015,10,27,33,16,1,1,1,0,0


In [27]:
# data types
db_founders_Encode.dtypes

id_founder                             int64
id_company                             int64
general_location                       int64
manager_exp                            int64
clevel_exp                             int64
engineering_exp                        int64
sales_exp                              int64
finance_exp                            int64
human_capital_exp                      int64
consulting_exp                         int64
founder_exp                            int64
foundation_year                        int64
years_of_experience_as_entreprenuer    int64
age_of_first_entrepreneurship          int64
age                                    int64
years_of_experience                    int64
number_of_startups                     int64
gender                                 int64
foreing_studies                        int64
foreing_work_experience                int64
foreing_founder                        int64
dtype: object

In [28]:
db_founders_Encode.isna().sum()

id_founder                             0
id_company                             0
general_location                       0
manager_exp                            0
clevel_exp                             0
engineering_exp                        0
sales_exp                              0
finance_exp                            0
human_capital_exp                      0
consulting_exp                         0
founder_exp                            0
foundation_year                        0
years_of_experience_as_entreprenuer    0
age_of_first_entrepreneurship          0
age                                    0
years_of_experience                    0
number_of_startups                     0
gender                                 0
foreing_studies                        0
foreing_work_experience                0
foreing_founder                        0
dtype: int64

In [29]:
db_founders_Encode.count()

id_founder                             640
id_company                             640
general_location                       640
manager_exp                            640
clevel_exp                             640
engineering_exp                        640
sales_exp                              640
finance_exp                            640
human_capital_exp                      640
consulting_exp                         640
founder_exp                            640
foundation_year                        640
years_of_experience_as_entreprenuer    640
age_of_first_entrepreneurship          640
age                                    640
years_of_experience                    640
number_of_startups                     640
gender                                 640
foreing_studies                        640
foreing_work_experience                640
foreing_founder                        640
dtype: int64

#Companies

In [141]:
# Query to connect with SQL data base
SQL_Query = pd.read_sql_query('''select * FROM "companies2" ''', conn)
companies_df = pd.DataFrame(SQL_Query)
companies_df.head(20)

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3.0,2017.0,0.0,2017.0,1400000.0,2017.0,1400000.0,48.0,33.0,8.0,3.0,1.0,4.0,0.0,0.0,0.0,2017.0,1.0,1400000.0,1400000.0,2017.0,1400000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,12.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,0.0,22.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7.0,2018.0,0.0,2018.0,10500.0,2018.0,10500.0,36.0,10.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,10500.0,10500.0,2018.0,10500.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10.0,2015.0,0.0,2015.0,200000.0,2015.0,200000.0,72.0,11.0,14.0,1.0,1.0,6.0,0.0,0.0,0.0,2015.0,1.0,200000.0,200000.0,2015.0,200000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23.0,2013.0,0.0,2013.0,150000.0,2021.0,40000000.0,0.0,1001.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2021.0,5.0,54800000.0,40000000.0,2021.0,40000000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51.0,2018.0,0.0,2018.0,20000.0,2018.0,20000.0,36.0,10.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,20000.0,20000.0,2018.0,20000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0
5,64.0,2017.0,0.0,2017.0,50000.0,2017.0,50000.0,48.0,10.0,3.0,1.0,1.0,4.0,0.0,0.0,0.0,2017.0,1.0,50000.0,50000.0,2017.0,50000.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,-2.0
6,93.0,2011.0,0.0,2011.0,30000.0,2011.0,30000.0,120.0,10.0,0.0,1.0,1.0,10.0,0.0,0.0,0.0,2011.0,1.0,30000.0,30000.0,2011.0,30000.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,6.0,-2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,1.0,0.0,7.0,-9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,-4.0
7,129.0,2020.0,0.0,2020.0,33000.0,2020.0,33000.0,12.0,10.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,2020.0,1.0,33000.0,33000.0,2020.0,33000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,1.0,1.0,25.0,20.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,1.0,1.0,25.0,20.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,1.0,1.0,25.0,20.0
8,141.0,2015.0,0.0,2015.0,250000.0,2021.0,15000000.0,0.0,130.0,98.0,0.0,1.0,0.0,0.0,1.0,1.0,2021.0,6.0,25500000.0,15000000.0,2021.0,15000000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.

In [142]:
# data types
companies_df.dtypes

id_company                                     float64
foundation_year                                float64
category                                       float64
seed_year                                      float64
seed_amount                                    float64
                                                ...   
mean_foreing_studies                           float64
mean_foreing_work_experience                   float64
mean_foreing_founder                           float64
mean_dp_years_of_experience                    float64
mean_dp_years_of_experience_as_entreprenuer    float64
Length: 67, dtype: object

In [143]:
companies_df.isna().sum()

id_company                                     0
foundation_year                                0
category                                       0
seed_year                                      0
seed_amount                                    0
                                              ..
mean_foreing_studies                           0
mean_foreing_work_experience                   0
mean_foreing_founder                           0
mean_dp_years_of_experience                    0
mean_dp_years_of_experience_as_entreprenuer    0
Length: 67, dtype: int64

In [144]:
companies_df.count()

id_company                                     510
foundation_year                                510
category                                       510
seed_year                                      510
seed_amount                                    510
                                              ... 
mean_foreing_studies                           510
mean_foreing_work_experience                   510
mean_foreing_founder                           510
mean_dp_years_of_experience                    510
mean_dp_years_of_experience_as_entreprenuer    510
Length: 67, dtype: int64

In [145]:
# Drop the null columns where all values are null
companies_df = companies_df.dropna(axis='columns', how='all')
companies_df.head()

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3.0,2017.0,0.0,2017.0,1400000.0,2017.0,1400000.0,48.0,33.0,8.0,3.0,1.0,4.0,0.0,0.0,0.0,2017.0,1.0,1400000.0,1400000.0,2017.0,1400000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,12.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,0.0,22.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7.0,2018.0,0.0,2018.0,10500.0,2018.0,10500.0,36.0,10.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,10500.0,10500.0,2018.0,10500.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10.0,2015.0,0.0,2015.0,200000.0,2015.0,200000.0,72.0,11.0,14.0,1.0,1.0,6.0,0.0,0.0,0.0,2015.0,1.0,200000.0,200000.0,2015.0,200000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23.0,2013.0,0.0,2013.0,150000.0,2021.0,40000000.0,0.0,1001.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2021.0,5.0,54800000.0,40000000.0,2021.0,40000000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51.0,2018.0,0.0,2018.0,20000.0,2018.0,20000.0,36.0,10.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2018.0,1.0,20000.0,20000.0,2018.0,20000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0


In [146]:
import re
# Encoding labels
le = LabelEncoder()
db_companies_Encode = companies_df.copy()

# Get the name of the columns
columnsName = db_companies_Encode.columns.tolist()
columnsName.remove('rounds_funding_factor')
r = re.compile("^mean")
columnsName = [column for column in columnsName if not r.match(column)]

# convert variables to numerical
for column in columnsName:
  db_companies_Encode[column] = pd.to_numeric(db_companies_Encode[column], errors='coerce').fillna(0).astype(np.int64)


db_companies_Encode.head()


,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3,2017,0,2017,1400000,2017,1400000,48,33,8,3,1,4.0,0,0,0,2017,1,1400000,1400000,2017,1400000,1,1,0,0,0,0,0,1,3,1,1,0,0,12,8,1,1,0,0,0,0,0,2,4,2,1,0,0,22,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,7.0
1,7,2018,0,2018,10500,2018,10500,36,10,0,2,0,3.0,0,0,0,2018,1,10500,10500,2018,10500,1,1,0,0,0,0,0,1,4,1,0,0,0,10,9,1,1,0,0,0,0,0,1,4,1,0,0,0,10,9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,10.0,9.0
2,10,2015,0,2015,200000,2015,200000,72,11,14,1,1,6.0,0,0,0,2015,1,200000,200000,2015,200000,1,1,0,0,0,0,0,1,1,1,0,0,0,12,2,1,1,0,0,0,0,0,1,1,1,0,0,0,12,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,12.0,2.0
3,23,2013,0,2013,150000,2021,40000000,0,1001,0,0,1,0.0,1,1,1,2021,5,54800000,40000000,2021,40000000,0,1,0,0,0,0,0,1,2,1,0,0,0,11,4,0,1,0,0,0,0,0,1,2,1,0,0,0,11,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,11.0,4.0
4,51,2018,0,2018,20000,2018,20000,36,10,0,1,0,3.0,0,0,0,2018,1,20000,20000,2018,20000,0,1,1,0,0,0,0,0,1,1,1,0,0,5,-1,0,1,1,0,0,0,0,0,1,1,1,0,0,5,-1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,5.0,-1.0


In [147]:
# data types
db_companies_Encode.dtypes

id_company                                       int64
foundation_year                                  int64
category                                         int64
seed_year                                        int64
seed_amount                                      int64
                                                ...   
mean_foreing_studies                           float64
mean_foreing_work_experience                   float64
mean_foreing_founder                           float64
mean_dp_years_of_experience                    float64
mean_dp_years_of_experience_as_entreprenuer    float64
Length: 67, dtype: object

In [148]:
db_companies_Encode.isna().sum()

id_company                                     0
foundation_year                                0
category                                       0
seed_year                                      0
seed_amount                                    0
                                              ..
mean_foreing_studies                           0
mean_foreing_work_experience                   0
mean_foreing_founder                           0
mean_dp_years_of_experience                    0
mean_dp_years_of_experience_as_entreprenuer    0
Length: 67, dtype: int64

In [149]:
db_companies_Encode.count()

id_company                                     510
foundation_year                                510
category                                       510
seed_year                                      510
seed_amount                                    510
                                              ... 
mean_foreing_studies                           510
mean_foreing_work_experience                   510
mean_foreing_founder                           510
mean_dp_years_of_experience                    510
mean_dp_years_of_experience_as_entreprenuer    510
Length: 67, dtype: int64

In [150]:
columnsName = db_companies_Encode.columns.tolist()
r = re.compile("^mean|^max")
columnsName = [column for column in columnsName if r.match(column)]
columnsName

['max_manager_exp',
 'max_clevel_exp',
 'max_engineering_exp',
 'max_sales_exp',
 'max_finance_exp',
 'max_human_capital_exp',
 'max_consulting_exp',
 'max_founder_exp',
 'max_number_of_startups',
 'max_gender',
 'max_foreing_studies',
 'max_foreing_work_experience',
 'max_foreing_founder',
 'max_dp_years_of_experience',
 'max_dp_years_of_experience_as_entreprenuer',
 'mean_manager_exp',
 'mean_clevel_exp',
 'mean_engineering_exp',
 'mean_sales_exp',
 'mean_finance_exp',
 'mean_human_capital_exp',
 'mean_consulting_exp',
 'mean_founder_exp',
 'mean_number_of_startups',
 'mean_gender',
 'mean_foreing_studies',
 'mean_foreing_work_experience',
 'mean_foreing_founder',
 'mean_dp_years_of_experience',
 'mean_dp_years_of_experience_as_entreprenuer']

Models


In [151]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint


In [152]:
for column in columnsName:
  db_companies_Encode[column] = db_companies_Encode.drop(column,axis='columns', inplace=False)

db_companies_Encode.head()

,id_company,foundation_year,category,seed_year,seed_amount,last_funding_year,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_year,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,sum_founder_exp,sum_number_of_startups,sum_gender,sum_foreing_studies,sum_foreing_work_experience,sum_foreing_founder,sum_dp_years_of_experience,sum_dp_years_of_experience_as_entreprenuer,mean_manager_exp,mean_clevel_exp,mean_engineering_exp,mean_sales_exp,mean_finance_exp,mean_human_capital_exp,mean_consulting_exp,mean_founder_exp,mean_number_of_startups,mean_gender,mean_foreing_studies,mean_foreing_work_experience,mean_foreing_founder,mean_dp_years_of_experience,mean_dp_years_of_experience_as_entreprenuer
0,3,2017,0,2017,1400000,2017,1400000,48,33,8,3,1,4.0,0,0,0,2017,1,1400000,1400000,2017,1400000,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1,1,0,0,0,0,0,2,4,2,1,0,0,22,15,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,7,2018,0,2018,10500,2018,10500,36,10,0,2,0,3.0,0,0,0,2018,1,10500,10500,2018,10500,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,1,1,0,0,0,0,0,1,4,1,0,0,0,10,9,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
2,10,2015,0,2015,200000,2015,200000,72,11,14,1,1,6.0,0,0,0,2015,1,200000,200000,2015,200000,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1,1,0,0,0,0,0,1,1,1,0,0,0,12,2,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
3,23,2013,0,2013,150000,2021,40000000,0,1001,0,0,1,0.0,1,1,1,2021,5,54800000,40000000,2021,40000000,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,0,1,0,0,0,0,0,1,2,1,0,0,0,11,4,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0
4,51,2018,0,2018,20000,2018,20000,36,10,0,1,0,3.0,0,0,0,2018,1,20000,20000,2018,20000,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,0,1,1,0,0,0,0,0,1,1,1,0,0,5,-1,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0


In [153]:
# Generate our categorical variable lists
db_companies_cat = ["foundation_year","seed_year","last_funding_year","dp_year"] #"max_manager_exp","max_clevel_exp","max_engineering_exp","max_sales_exp","max_finance_exp","max_human_capital_exp","max_consulting_exp","max_founder_exp","max_number_of_startups","max_gender","max_foreing_studies","max_foreing_work_experience","max_foreing_founder"]
db_companies_cat

['foundation_year', 'seed_year', 'last_funding_year', 'dp_year']

In [154]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(db_companies_Encode[db_companies_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(db_companies_cat)
encode_df.head()

,foundation_year_2000,foundation_year_2004,foundation_year_2006,foundation_year_2008,foundation_year_2009,foundation_year_2010,foundation_year_2011,foundation_year_2012,foundation_year_2013,foundation_year_2014,foundation_year_2015,foundation_year_2016,foundation_year_2017,foundation_year_2018,foundation_year_2019,foundation_year_2020,foundation_year_2021,seed_year_2000,seed_year_2004,seed_year_2006,seed_year_2008,seed_year_2009,seed_year_2010,seed_year_2011,seed_year_2012,seed_year_2013,seed_year_2014,seed_year_2015,seed_year_2016,seed_year_2017,seed_year_2018,seed_year_2019,seed_year_2020,seed_year_2021,last_funding_year_2000,last_funding_year_2008,last_funding_year_2011,last_funding_year_2012,last_funding_year_2013,last_funding_year_2014,last_funding_year_2015,last_funding_year_2016,last_funding_year_2017,last_funding_year_2018,last_funding_year_2019,last_funding_year_2020,last_funding_year_2021,dp_year_0,dp_year_2000,dp_year_2008,dp_year_2011,dp_year_2012,dp_year_2013,dp_year_2014,dp_year_2015,dp_year_2016,dp_year_2017,dp_year_2018,dp_year_2019,dp_year_2020,dp_year_2021
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [155]:
# Merge one-hot encoded features and drop the originals
db_companies_Encode = db_companies_Encode.merge(encode_df,left_index=True, right_index=True)
db_companies_Encode = db_companies_Encode.drop(db_companies_cat,1)
db_companies_Encode.head()

,id_company,category,seed_amount,amount_last_funding,month_since_last_funding,number_employees,increase_employees,number_investors,current_status,rounds_funding_factor,soonicorn,more50_employees,more100_employees,dp_rounds_count,dp_rounds_amount_sum,dp_rounds_amount_max,dp_last_round_year,dp_last_round_amount,max_manager_exp,max_clevel_exp,max_engineering_exp,max_sales_exp,max_finance_exp,max_human_capital_exp,max_consulting_exp,max_founder_exp,max_number_of_startups,max_gender,max_foreing_studies,max_foreing_work_experience,max_foreing_founder,max_dp_years_of_experience,max_dp_years_of_experience_as_entreprenuer,sum_manager_exp,sum_clevel_exp,sum_engineering_exp,sum_sales_exp,sum_finance_exp,sum_human_capital_exp,sum_consulting_exp,...,seed_year_2009,seed_year_2010,seed_year_2011,seed_year_2012,seed_year_2013,seed_year_2014,seed_year_2015,seed_year_2016,seed_year_2017,seed_year_2018,seed_year_2019,seed_year_2020,seed_year_2021,last_funding_year_2000,last_funding_year_2008,last_funding_year_2011,last_funding_year_2012,last_funding_year_2013,last_funding_year_2014,last_funding_year_2015,last_funding_year_2016,last_funding_year_2017,last_funding_year_2018,last_funding_year_2019,last_funding_year_2020,last_funding_year_2021,dp_year_0,dp_year_2000,dp_year_2008,dp_year_2011,dp_year_2012,dp_year_2013,dp_year_2014,dp_year_2015,dp_year_2016,dp_year_2017,dp_year_2018,dp_year_2019,dp_year_2020,dp_year_2021
0,3,0,1400000,1400000,48,33,8,3,1,4.0,0,0,0,1,1400000,1400000,2017,1400000,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,7,0,10500,10500,36,10,0,2,0,3.0,0,0,0,1,10500,10500,2018,10500,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10,0,200000,200000,72,11,14,1,1,6.0,0,0,0,1,200000,200000,2015,200000,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23,0,150000,40000000,0,1001,0,0,1,0.0,1,1,1,5,54800000,40000000,2021,40000000,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,51,0,20000,20000,36,10,0,1,0,3.0,0,0,0,1,20000,20000,2018,20000,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [156]:
db_companies_Encode.dtypes

id_company                    int64
category                      int64
seed_amount                   int64
amount_last_funding           int64
month_since_last_funding      int64
                             ...   
dp_year_2017                float64
dp_year_2018                float64
dp_year_2019                float64
dp_year_2020                float64
dp_year_2021                float64
Length: 124, dtype: object

Model 50 employees

In [157]:
# Split our preprocessed data into our features and target arrays
y = db_companies_Encode["more50_employees"].values
X = db_companies_Encode.drop(["more50_employees"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 80)                9920      
                                                                 
 dense_34 (Dense)            (None, 30)                2430      
                                                                 
 dense_35 (Dense)            (None, 1)                 31        
                                                                 
Total params: 12,381
Trainable params: 12,381
Non-trainable params: 0
_________________________________________________________________


In [158]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [159]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [160]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [161]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
12/12 [==============================] - 0s 2ms/step - loss: 136272.5156 - accuracy: 0.3298
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 81146.9766 - accuracy: 0.4319
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 32154.3066 - accuracy: 0.6126
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 72495.0156 - accuracy: 0.5445
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 50612.3359 - accuracy: 0.5942
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 41048.2891 - accuracy: 0.6702
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 41258.0820 - accuracy: 0.6466
Epoch 8/100
12/12 [==============================] - 0s 2ms/step - loss: 14179.8936 - accuracy: 0.7696
Epoch 9/100
12/12 [==============================] - 0s 2ms/step - loss: 13977.0537 - accuracy: 0.7880
Epoch 10/100
12/12 [==============================] - 0s 2ms/step - loss

In [162]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 66892.3906 - accuracy: 0.8438 - 106ms/epoch - 27ms/step
Loss: 66892.390625, Accuracy: 0.84375


Model 100 employees

In [163]:
# Split our preprocessed data into our features and target arrays
y = db_companies_Encode["more100_employees"].values
X = db_companies_Encode.drop(["more100_employees"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 80)                9920      
                                                                 
 dense_37 (Dense)            (None, 30)                2430      
                                                                 
 dense_38 (Dense)            (None, 1)                 31        
                                                                 
Total params: 12,381
Trainable params: 12,381
Non-trainable params: 0
_________________________________________________________________


In [164]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [165]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
12/12 [==============================] - 0s 2ms/step - loss: 213304.8594 - accuracy: 0.2330
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 108061.4062 - accuracy: 0.6623
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 75284.7734 - accuracy: 0.5052
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 42538.4219 - accuracy: 0.7775
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 34363.4727 - accuracy: 0.6754
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 21392.9199 - accuracy: 0.7592
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 29062.8691 - accuracy: 0.7408
Epoch 8/100
12/12 [==============================] - 0s 2ms/step - loss: 11181.1680 - accuracy: 0.7827
Epoch 9/100
12/12 [==============================] - 0s 3ms/step - loss: 21908.7812 - accuracy: 0.7984
Epoch 10/100
12/12 [==============================] - 0s 2ms/step - los

In [166]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 47214.9883 - accuracy: 0.8359 - 106ms/epoch - 26ms/step
Loss: 47214.98828125, Accuracy: 0.8359375


Model soonicorn

In [167]:
# Split our preprocessed data into our features and target arrays
y = db_companies_Encode["soonicorn"].values
X = db_companies_Encode.drop(["soonicorn"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 80)                9920      
                                                                 
 dense_40 (Dense)            (None, 30)                2430      
                                                                 
 dense_41 (Dense)            (None, 1)                 31        
                                                                 
Total params: 12,381
Trainable params: 12,381
Non-trainable params: 0
_________________________________________________________________


In [168]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [169]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
12/12 [==============================] - 0s 2ms/step - loss: 602171.5000 - accuracy: 0.6754
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 172186.8750 - accuracy: 0.1571
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 76822.2656 - accuracy: 0.6702
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 30784.3398 - accuracy: 0.5838
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 44735.3398 - accuracy: 0.6492
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 29594.8359 - accuracy: 0.8377
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 69861.5547 - accuracy: 0.4895
Epoch 8/100
12/12 [==============================] - 0s 2ms/step - loss: 35224.6719 - accuracy: 0.7225
Epoch 9/100
12/12 [==============================] - 0s 2ms/step - loss: 26100.7402 - accuracy: 0.7251
Epoch 10/100
12/12 [==============================] - 0s 2ms/step - los

In [170]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 24480.8516 - accuracy: 0.8438 - 92ms/epoch - 23ms/step
Loss: 24480.8515625, Accuracy: 0.84375


File


In [ ]:
from google.colab import files

with open('/content/drive/MyDrive/Final_project/Second_Deliverable/key_config.txt', "r") as file:
    FileasList = file.readlines()
    username = FileasList[0]
    password = FileasList[1]

    new_line = []
    # Loop through the file
    for i in FileasList:
        # Get rid of any new lines ("enter")
        i = i.replace("\n", "")
        # Add the cleaned data to the new_line list
        new_line.append(i)
    # Set the username to be the first object
    username = new_line[0]
    # Set the password to be the second object
    password = new_line[1]